# Import libraries

In [21]:
# Data analysis
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt

# Machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score

# Loading and Pre-processing data

In [22]:
# Review dataframe
insurance = pd.read_csv('insurance_data.csv', sep=';')
print(insurance.shape)
insurance.head(5)

(5500, 15)


,insuree#,gender,is45OrOlder,isMarried,hasKids,insuredMonths,termLifeInsurance,multipleTermLifePolicies,healthInsurance,healthRiders,premiumFrequency,eStatements,monthlyPremium,totalPremium,renewal
0,1,F,0.0,Yes,Yes,23,Yes,No,No,NaN,12.0,Yes,"19,65","451,55",Y
1,2,F,1.0,No,No,42,Yes,Yes,Class A,3,1.0,Yes,"84,65","3541,35",N
2,3,F,0.0,Yes,No,72,Yes,No,No,NaN,12.0,No,"19,4","1496,45",Y
3,4,F,0.0,Yes,Yes,13,Yes,No,No,NaN,12.0,No,"19,55","265,3",Y
4,5,F,0.0,No,No,37,Yes,Yes,Class A,"3,4",1.0,No,"100,3","3541,4",Y


In [31]:
# Check NA fields
insurance.isna().sum().sort_values(ascending=False)

healthRiders                1982
premiumFrequency             114
gender                       113
is45OrOlder                   96
insuree#                       0
isMarried                      0
hasKids                        0
insuredMonths                  0
termLifeInsurance              0
multipleTermLifePolicies       0
healthInsurance                0
eStatements                    0
monthlyPremium                 0
totalPremium                   0
renewal                        0
dtype: int64

In [32]:
# Value counts of all fields with NA
df = insurance.copy()
for i in df.columns[df.isnull().any()]:
    print(df[i].value_counts())

M    2719
F    2668
Name: gender, dtype: int64
0.0    4525
1.0     879
Name: is45OrOlder, dtype: int64
3,4        474
2,3,4      434
1,2,3,4    374
2          304
1          277
1,2        247
3          238
4          232
1,3,4      228
2,3        139
2,4        133
1,2,4      131
1,2,3      126
1,4         95
1,3         86
Name: healthRiders, dtype: int64
1.0     2959
12.0    1308
3.0     1119
Name: premiumFrequency, dtype: int64


In [39]:
## Dealing with NA Values

# Gender: Remove all NAs as there is no way for us to fill an arbitrary gender
df = df[df.gender.notna()]

# is45OrOlder: Remove all NAs as there is no way for us to fill an arbitrary age
df = df[df.is45OrOlder.notna()]

# healthRiders: change all values to 0 as there may be people with policies without riders
df.loc[df.healthRiders.isna()] = 0

# premiumFrequency: change all values to 0 as premium can be fully paid
df.loc[df.premiumFrequency.isna()] = 0

df.isna().sum().sort_values()

insuree#                    0
gender                      0
is45OrOlder                 0
isMarried                   0
hasKids                     0
insuredMonths               0
termLifeInsurance           0
multipleTermLifePolicies    0
healthInsurance             0
healthRiders                0
premiumFrequency            0
eStatements                 0
monthlyPremium              0
totalPremium                0
renewal                     0
count_riders                0
dtype: int64

In [40]:
# Change all yes no into 0-1
df.replace('Yes', 1).replace('No', 0)

,insuree#,gender,is45OrOlder,isMarried,hasKids,insuredMonths,termLifeInsurance,multipleTermLifePolicies,healthInsurance,healthRiders,premiumFrequency,eStatements,monthlyPremium,totalPremium,renewal,count_riders
0,0,0,0.0,0,0,0,0,0,0,0,0.0,0,0,0,0,0
1,2,F,1.0,0,0,42,1,1,Class A,3,1.0,1,"84,65","3541,35",N,1
2,0,0,0.0,0,0,0,0,0,0,0,0.0,0,0,0,0,0
3,0,0,0.0,0,0,0,0,0,0,0,0.0,0,0,0,0,0
4,5,F,0.0,0,0,37,1,1,Class A,"3,4",1.0,0,"100,3","3541,4",Y,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5495,5496,F,0.0,1,1,4,1,1,Class A,4,1.0,1,"85,95","381,3",N,1
5496,0,0,0.0,0,0,0,0,0,0,0,0.0,0,0,0,0,0
5497,5498,M,0.0,1,0,56,1,0,Class A,4,3.0,1,"80,9","4557,5",Y,1
5498,0,0,0.0,0,0,0,0,0,0,0,0.0,0,0,0,0,0


In [41]:
non_numeric_fields = df.select_dtypes(exclude = ['int64', 'float64']).columns.tolist()
print(non_numeric_fields)

['gender', 'isMarried', 'hasKids', 'termLifeInsurance', 'multipleTermLifePolicies', 'healthInsurance', 'healthRiders', 'eStatements', 'monthlyPremium', 'totalPremium', 'renewal']


In [42]:
# avg no. health riders + which is most popular 
def count_riders(x):
    if isinstance(x, str) == True:
        return x.count(',')+1
    else:
        if x == 0:
            return 0
        return 1
df['count_riders'] = df.healthRiders.apply(lambda x: count_riders(x))
df.head(20)

# count number of policies (rmb 0 means none)
# df[['healthriders1','healthriders2', 'healthriders3', 'healthriders4']] = df.healthRiders.str.split(',', expand = True) 

,insuree#,gender,is45OrOlder,isMarried,hasKids,insuredMonths,termLifeInsurance,multipleTermLifePolicies,healthInsurance,healthRiders,premiumFrequency,eStatements,monthlyPremium,totalPremium,renewal,count_riders
0,0,0,0.0,0,0,0,0,0,0,0,0.0,0,0,0,0,0
1,2,F,1.0,No,No,42,Yes,Yes,Class A,3,1.0,Yes,"84,65","3541,35",N,1
2,0,0,0.0,0,0,0,0,0,0,0,0.0,0,0,0,0,0
3,0,0,0.0,0,0,0,0,0,0,0,0.0,0,0,0,0,0
4,5,F,0.0,No,No,37,Yes,Yes,Class A,"3,4",1.0,No,"100,3","3541,4",Y,2
5,6,M,1.0,No,No,3,Yes,Yes,Class A,2,1.0,Yes,"81,35","233,7",N,1
6,0,0,0.0,0,0,0,0,0,0,0,0.0,0,0,0,0,0
7,8,M,0.0,No,No,24,Yes,No,Class A,2,1.0,Yes,"85,95","2107,15",Y,1
8,9,F,0.0,No,No,24,Yes,No,Class B,1,1.0,No,"56,35","1381,2",Y,1
9,10,F,1.0,No,No,31,Yes,Yes,Class A,"2,3,4",1.0,Yes,"99,95","3186,65",N,3
